---
title       : "Machine Learning in Julia"
subtitle    :
author      : Paul Schrimpf
date        : `j using Dates; print(Dates.today())`
bibliography: "../ml.bib"
---

[![](https://i.creativecommons.org/l/by-sa/4.0/88x31.png)](http://creativecommons.org/licenses/by-sa/4.0/)

This work is licensed under a [Creative Commons Attribution-ShareAlike
4.0 International
License](http://creativecommons.org/licenses/by-sa/4.0/) 


### About this document 

This document was created using Weave.jl. The code is available in
[on github](https://github.com/schrimpf/NeuralNetworkEconomics.jl). The same
document generates both static webpages and associated (jupyter
notebook)[ml-julia.ipynb]. 

$$
\def\indep{\perp\!\!\!\perp}
\def\Er{\mathrm{E}}
\def\R{\mathbb{R}}
\def\En{{\mathbb{E}_n}}
\def\Pr{\mathrm{P}}
\newcommand{\norm}[1]{\left\Vert {#1} \right\Vert}
\newcommand{\abs}[1]{\left\vert {#1} \right\vert}
\DeclareMathOperator*{\argmax}{arg\,max}
\DeclareMathOperator*{\argmin}{arg\,min}
$$

In [ ]:
markdown = try
  "md" in keys(WEAVE_ARGS) && WEAVE_ARGS["md"]
catch
  false
end

if !("DISPLAY" ∈ keys(ENV))
  # Make gr and pyplot backends for Plots work without a DISPLAY
  ENV["GKSwstype"]="nul"
  ENV["MPLBACKEND"]="Agg"
end

using NeuralNetworkEconomics
docdir = joinpath(dirname(Base.pathof(NeuralNetworkEconomics)), "..","docs")

using Pkg
Pkg.activate(docdir)
Pkg.instantiate()

# Introduction

This document is a companion to my ["Machine learning in
economics"](ml-intro.md). Those notes discuss the recent use of
machine learning in economics, with a focus on lasso and random
forests. The code in those notes is written in R. This document will
look at similar code in Julia.

# RCall

If you want to use the methods of Chernozhukov and coauthors
implements in the R packaga @hdm or the methods of Athey and coauthors
implemented in the R package @grf , then it makes sense to use the R
pacakge. You could simply write all your code in R. However, if you
prefer using Julia, you can just call the necessary R functions with
[`RCall.jl`](https://github.com/JuliaInterop/RCall.jl). 


Here, we load the pipeline data used in the [machine
learning methods notes](ml-methods.md), and do some cleaning in Julia.

In [ ]:
using RCall, DataFrames, Missings, Statistics
R"load(paste($(docdir),\"/rmd/pipelines.Rdata\",sep=\"\"))"
println(R"ls()")
data = @rget data # data on left is new Julia variable, data on right is the one in R
println(R"summary(data)")
println(describe(data))
for c in 59:107 # columns of state mileage, want missing->0
  replace!(x->(ismissing(x) || isnan(x)) ? 0.0 : x, data[c])
end
println(describe(data[:,59:107]))

Suppose we want to estimate the coefficient on `transPlant` (capital)
in a partially linear model with `transProfit` (profit) as the
outcome. This can be done with the R function `hdm::rlassoEffects`.

In [ ]:
R"library(hdm)"
completedata = dropmissing(data,[1:10..., 59:122...], disallowmissing=true)
y = completedata[:transProfit]
inc = .!isnan.(y)
y = y[inc]
X = completedata[inc,[6:7..., 59:121...]]
cols = [std(X[c])>0 for c in 1:ncol(X)]
X = X[:,cols]
est = R"rlassoEffects($(X), $(y), index=c(1:2))"
R"summary($est)"

# MLJ.jl

[`MLJ.jl`](https://github.com/alan-turing-institute/MLJ.jl) is a machine
learning framework for Julia. It gives a unified interface for many
machine learning algorithms and tasks. Similar R packages include
`caret` and `MLR`. [`scikit-learn`](https://scikit-learn.org/stable/)
is a similar Python package. 

For more information on MLJ see 

- [`MLJ.jl docs`](https://alan-turing-institute.github.io/MLJ.jl/stable/)

- [MLJ
  tutorials](https://alan-turing-institute.github.io/MLJTutorials/)

You can see a list of models registered to work with `MLJ.jl` on
[github](https://github.com/alan-turing-institute/MLJModels.jl/blob/master/src/registry/Models.toml),
or by calling `MLJ::models()`.

In [ ]:
using MLJ
models()

To use these models, you need the corresponding package to be
installed and loaded. The `@load` macro will load the needed package(s)
for any model.

In [ ]:
lasso_model = @load LassoRegressor pkg=MLJLinearModels

Let's fit lasso to the same pipeline data as above.

In [ ]:
lasso_model.lambda = 1.0
lasso = machine(lasso_model, X, y)
train,test = partition(eachindex(y), 0.6, shuffle=true)
fit!(lasso, rows=train)
yhat = predict(lasso, rows=test)
println(yhat[1:10])
println("MSE/var(y) = $(mean((y[test].-yhat).^2)/var(y[test]))")

That doesn't look very good. All the predictions are zero. This could
happen when the regularization parameter, `lambda`, is too
large. However, in this case the problem is something else. The
warning messages indicate numeric problems when minimizing the lasso
objective function. This can happen when `X` is poorly scaled. The
algorithm used to compute the lasso estimates works best when the
coefficients are all roughly the same scale. The existing `X`'s have
wildly different scales, which causes problems. This situation is
common, so `MLJ.jl` has functions to standardize variables. It is
likely that the `hdm` package in R does something similar internally.

In [ ]:
lasso_stdx = @pipeline PipeLasso(std=Standardizer(),
                  lasso=LassoRegressor(lambda=1.0*std(y),
                                       solver=MLJLinearModels.ISTA(max_iter=10000)
                                       )
                                 )
m = machine(lasso_stdx, X, y)
fit!(m, rows=train)
yhat = predict(m , rows=test)
println("MSE/var(y) = $(mean((y[test].-yhat).^2)/var(y[test]))")

# Get the fitted coefficients
coef = fitted_params(m).fitted_params[1].coefs
intercept = fitted_params(m).fitted_params[1].intercept
sum(abs.(coef).>1e-8) # number non-zero

If we want to tune `lambda` using cross-validation, we can use the
`range` and `TunedModel` functions.

In [ ]:
r = range(lasso_stdx, :(lasso.lambda), lower=1e1, upper=1e10, scale=:log)
t=TunedModel(model=lasso_stdx,
             resampling=CV(nfolds=5),
             tuning=Grid(resolution=10), 
             ranges=r,
             measure=rms)
m = machine(t, X, y)
fit!(m, rows=train, verbosity=1)
yhat = predict(m , rows=test)
println("MSE/var(y) = $(mean((y[test].-yhat).^2)/var(y[test]))")

In [ ]:
using Plots
cvmse = m.report.measurements
λ = Float64.(m.report.parameter_values[:])
plot(λ, cvmse, xlab="λ", ylab="CV(MSE)")

# Flux.jl 

[`Flux.jl`](https://fluxml.ai/Flux.jl/stable/) is another Julia
package for machine learning. It seems to be emerging as the leading
Julia package for neural networks and deep learning, but other machine
learning models can also be implemented using `Flux.jl`. 

Let's create a lasso model in `Flux.jl`.

In [ ]:
using Flux
# Scale the variables
Xstd = Flux.normalise(Matrix(X))
X_train = Xstd[train,:]
X_test = Xstd[test,:]
yscale = std(y)
ymean = mean(y)
ystd = (y .- ymean)./yscale
y_train = ystd[train]
y_test = ystd[test]

# Set up the model parameters and initial values
βols = (X_train'*X_train) \ (X_train'*(y_train .- mean(y_train)))
β = param(zeros(ncol(X))) #βols) #zeros(ncol(X)))
b = param([mean(y_train)])
θ = Tracker.Params([β,b])

# Define the loss function
ψ = ones(length(β))
λ = 2.0
pred(x) = b .+ x*β
mse(x,y) = mean( (pred(x) .- y).^2 )
penalty(y) = λ/length(y)*norm(ψ.*β,1)
loss(x,y) =  mse(x,y) + penalty(y)
@show loss(X_train,y_train)

# minimize loss
maxiter=2000
obj = zeros(maxiter)
mse_train = zeros(maxiter)
mse_test = zeros(maxiter)
for i in 1:maxiter
  Flux.train!(loss, θ, [(X_train, y_train)], Flux.AMSGrad())
  mse_train[i] = Tracker.data(mse(X_train,y_train))
  mse_test[i] = Tracker.data(mse(X_test, y_test))
  obj[i] = Tracker.data(loss(X_train,y_train))
end
lo = 1
hi = 250
plot(obj[lo:hi], ylab="Loss=MSE + λ/n*||β||₁", xlab="Iteration")

In [ ]:
plot(lo:hi, [mse_train[lo:hi] mse_test[lo:hi]], ylab="MSE", xaxis=("Iteration")
     , lab=["Train" "Test"])

The minimization methods in `Flux.train!` are all variants of gradient
descent. Each call to `Flux.train!` runs one iteration of the
specified solver. To find a locaol minimum, `Flux.train!` can be
called repeatedly until progress stops. The above loop is a simple way
to do this. The `@epoch` macro can also be useful. 

Gradient descent works well for neural networks, but are is ideal for
Lasso. Without further adjustment, gradient descent gets stuck in a
cycle as jumps from one side of the other of the absolute value in the
lasso penalty. Nonetheless, the results are near the true minimum,
even though it never exactly gets there. 


# Additional Resources

- @klok2019 *Statistics with Julia:Fundamentals for Data Science,
  MachineLearning and Artificial Intelligence*